# Exercise 44: Electricity storage 

In this exercise you will work an extension of the $\texttt{mBasicInt}$ model, with the extension being the inclusion of a technology with capacity for electricity storage. The analytical structure of the model is outlined in Exercise 44_H, which is available on absalon. We strongly recommend that you cover the material in Exercise 44-H, before proceeding.

We start by importing basic packages and specifying where to place figures.

In [1]:
%run ../stdPackages.ipynb
from pyDbs import *
from IPython.display import Image
figs_path = os.path.join(d['curr'],'Figs')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The data we are using to parameterize the model is located at

In [2]:
path_to_data = os.path.join(d['curr'],'E44','Data','E44_Data.xlsx')
print(path_to_data)

c:\Users\Peter Bendix\OneDrive\Skrivebord\EnergyEconomicsE2024\Exercises\E44\Data\E44_Data.xlsx


The table below translates the names used in the data to the symbols used in the model. 

| Name in excel file | Model symbol |
|:--------|:--------:|
| id | $i$ | 
| BFt | $j$ |
| h | $h$ |
|  FuelMix   |  $\mu_{i,j}$   |
|  GeneratingCapacity   | $q_i$   |
|  OtherMC   | $c_i^{oth}$   |
| FuelPrice | $p_j^f$ |
| EmissionIntensity | $\phi_j$ |
| MWP | $\mu$ |
| LoadVariation $\times$ Load | $D_h$ |
| CapVariation | $\gamma_{i,h} $ |
| sCap | $\overline{S}_i$ |
| effS | $\sqrt{\eta_i}$ |

Note that $c$ is also an index for the consumer type in the data file. 

The following section imports the data:

In [3]:
kwargs = {'variables': ['Fundamentals', 'Load', 'Generators_Other'],
        'maps': ['Generators_Categories','hMaps'],
        'variable2D': ['HourlyVariation','Generators_FuelMix'],
        'scalars': ['Scalars']}

db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

# E1

Recall that the social planner problem in the system of equations in __(1)__ of Exercise 44-H is a linear programming problem an can be expressed on the form

\begin{align} 
    &\min_{x} \ c^T\cdot x \tag{2A}\\ 
    &l\leq x\leq u, \tag{2B} \\
    &A_{eq}\times x  = b_{eq} \tag{2C}\\ 
\end{align}
where: 
* $x$ is the vector of choice variables of length ($N$).
* $c, l, u$ are coefficient vectors of the same length ($N$).
* $b_{eq}, b_{ub}$ are coefficient vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.


In the `py`-folder for this exercise, the modelclass we will be working with is located in `ModelClass.py`. Under the `mSimple` class (post line 91), you will fill in the missing lines of code to extend the $\texttt{mBasicInt}$ model to include energy storage. 


_c-vector:_  
__For the c-vector in (2A) fill in coefficient related to the discharge $(Y_{i,h}^d)$ and charge $(Y_{i,h}^c)$ decisions in the c-property of__ `mSimple`:
```python
    @property
    def c(self):
        return [{'varName': 'Generation',
                 'value': adjMultiIndex.bc(self.db['mc'], self.globalDomains['Generation']), 
                 'conditions': getTechs(['Standard'],self.db)},
                {'varName': 'HourlyDemand',
                 'value':-adjMultiIndex.bc(self.db['MWP'], self.globalDomains['HourlyDemand'])},
                {'varName': 'discharge',
                 'value': ...Fill in here...,
                 'conditions': getTechs('Storage',self.db)},
                {'varName': 'charge',
                 'value': ...Fill in here...,
                 'conditions': ...Fill in here...}]
```
_u-vector:_   
__For the u-vector in (2B) fill in upper bound related to the discharge $(Y_{i,h}^d)$, charge $(Y_{i,h}^c)$, and storage $(S_{i,h})$ decisions in the u-property of `mSimple`:__
```python
    @property
    def u(self):
        return [{'varName': 'Generation', 
                 'value': adjMultiIndex.bc(self.hourlyGeneratingCap, self.globalDomains['Generation']), 
                 'conditions': getTechs(['Standard'],self.db)},
                {'varName': 'HourlyDemand', 'value': self.hourlyLoad_c},
                {'varName': 'stored', 
                 'value': adjMultiIndex.bc(self.db['sCap'], self.globalDomains['stored'])},
                {'varName': 'discharge', 
                 'value': ...Fill in here...,
                 'conditions': getTechs('Storage',self.db)},
                {'varName': 'charge',
                 'value': Fill in here..., 
                 'conditions': Fill in here...}]
```

__Finally, make sure your code is correct by solving for the baseline scenario, where this is no storage__:

In [5]:
import sys
sys.path.append(os.path.join(d['curr'], 'E44', 'py'))
from ModelClass_SolutionGuide import *
m = mSimple(db)
m()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


Save solution for later:

In [ ]:
base_db = m.db.copy()

In the following, we will aggregate the fuel types to the following types:

In [ ]:
agg_BFt = {
    'Fueloil':'Oil','Gasoil':'Oil', 
    'Straw':'Biomass','WoodPellets':'Biomass','WoodChips':'Biomass','WoodWaste':'Biomass',
    'Biogas':'Other Bio','BioOil':'Other Bio',
    'Coal':'Coal',
    'NatGas':'Natural gas',
    'Waste':'Waste'
}

___
___Answer:___

See the ModelClass_SolutionGuide.py.

## E2

Below, the average merit order curve, the average marginal inverse fuel effiency, and the average marginal $CO_2$ intensity is illustrated.

**Describe the supply curve.**

In [ ]:
Image(os.path.join(figs_path,'E2.2_sol.png'))

_Note: The code for generating the figure is:_

In [ ]:
%%capture
# Settings
ϵ=np.finfo(float).eps*1e10
lower_point=0
upper_point=m.db['MWP']*1.1

# Make dataframe
df_fuelineff = m.db['FuelMix'].rename(agg_BFt,level='BFt').groupby(['id','BFt']).sum().unstack('BFt').rename_axis(None,axis=1)
df = pd.concat(
    [m.hourlyGeneratingCap.groupby('id').agg('mean').rename('q').astype(float),m.db['mc'].rename('p').astype(float)]+
    [df_fuelineff]+
    [plantEmissionIntensity(m.db).xs('CO2',level='EmissionType').rename('m').mul(1000)]
    ,axis=1
).fillna(0)

# Sort and aggregate
df = df.sort_values(['p'])
df['q_cs'] = df['q'].transform('cumsum')

# Add upper points for step plotting
df_up = df.copy()
df_up['q_cs'] = df_up['q_cs']+ϵ
df_up['p'] = df_up['p'].shift(-1).fillna(upper_point)
df_up[df_fuelineff.columns.values] = df_up[df_fuelineff.columns.values].shift(-1)
df_up['m'] = df_up['m'].shift(-1)
df = pd.concat([df,df_up],axis=0).sort_values(['p','q_cs'])
df['q_diff'] = df['q_cs'].diff().div(2)
df['q_mean'] = df['q_cs']-df['q_diff']

# Now plot
fig, ax = plt.subplots(ncols=1,nrows=3,sharex=True,sharey=False)

# Merit order cuve:
df.set_index(['q_cs'])['p'].plot(ax=ax[0],ylabel=r'$€/$GJ',title='Average merit order curve')
for i in adj.rc_pd(m.db['id'],getTechs(['Standard'],m.db)):
    df_tmp = df.loc[i].copy()
    ax[0].text(x=df_tmp.iloc[0]['q_mean'],y=df_tmp.iloc[0]['p']-1,s=i,ha='center',va='top')

# Marginal fuel inefficiency
df.set_index(['q_cs'])[df_fuelineff.columns].plot(kind='area',linewidth=0,ax=ax[1],legend=False,stacked=True,ylabel='GJ input/GJ output',title='Average marginal inverse fuel efficiency')
ax[1].set_ylim(top=2)
fig.legend(handles=ax[1].get_legend_handles_labels()[0],loc='upper center', ncol=len(df_fuelineff.columns), frameon=True, bbox_to_anchor=(0.55,0.64))

# Marginal emission coefficient
df.set_index(['q_cs'])['m'].plot(kind='area',linewidth=0,ax=ax[2],legend=False,xlabel='GJ output',ylabel=r'kg CO$_2/$GJ',title=r'Average marginal CO$_2$ intensity')

fig.tight_layout()
fig.savefig(f"{figs_path}\\E2.2_sol.png", edgecolor='k',bbox_inches='tight')

___
___Answer:___

- `id6` is a wind farm with a marginal costs of generation of 3 $€/$GJ. The installed/theoretical capacity if 25 but the average/practical generating capacity is only around 7.5.
- `id4` is a relatively cheap waste incineration plant, which is associted with CO$_2$ emissions.
- `id1` is next on the merit order curve and is predominately a coal plant. Even though it is not the most fuel inefficient generator it has by far the largest emission intensity.
- `id2` is a predominately a natural-gas fired plant and has the second highest emission intensity.
- `id3` is a biomass-fired plant associated with moderate CO$_2$ emissions since it also uses a small input of coal.
- `id5` is the last generator on the merit order curve and it is relatively costly mainly due to high fuel costs. 

## E3

**Describe the residual demand curve and the prices.**

In [ ]:
Image(os.path.join(figs_path,'E2.3_sol.png'))

_Note: The code for generating the figure is:_

In [ ]:
%%capture
# Make dataframe
int_id = adj.rc_pd(m.db['id2hvt'], pd.Index(['Wind','Solar'], name = 'hvt')).droplevel('hvt')
df_sort =  pd.concat([
    adj.rc_pd(m.hourlyGeneratingCap,int_id).groupby('h').sum().rename('E').astype(float),
    m.hourlyLoad.rename('L').astype(float),
    m.db['marginalSystemCosts'].rename('p').astype(float)
],axis=1)

# Calculate residual demand
df_sort['resd'] = df_sort['L']-df_sort['E']

# Sort dataframe
df_sort = df_sort.sort_values(['resd'],ascending=False).reset_index()
df_sort.index = df_sort.index/len(df_sort)

# Make new dataframe with time mappings
df_h = df_sort.merge(m.db['h2HourOfTheDay'].to_frame(index=False))

# Aggregate for average hour of the day
df_h = df_h.groupby('HourOfTheDay')[['resd','p']].mean()

# Make plot
fig, ax = plt.subplots(nrows=2,ncols=2,sharey='row',sharex='col')

df_sort['resd'].plot(ax=ax[0,0],ylabel=r'Residual demand (GJ)')
df_sort['p'].plot(ax=ax[1,0],ylabel=r'Marginal system costs ($€/$GJ)',xlabel='Capacity factor')
df_h['resd'].plot(ax=ax[0,1])
df_h['p'].plot(xlabel='Average hour of the day (GJ)',ax=ax[1,1])

fig.tight_layout()
fig.savefig(f"{figs_path}\\E2.3_sol.png", edgecolor='k',bbox_inches='tight')

___
___Answer:___

In the baseline scenario - where this is no capacity for electricity storage - there is a surplus of wind electricity in around 7% of the year (upper left figure). This surplus electricity gets curtailed in equilibrium. Comparing the upper left panel with the lower left figure, we see that prices of the year closely follows the share of the residual demand curve. 

This is also illustrated in the right column of the figure, which shows the average residual demand curve (upper right) and the marginal system costs (lower right) over the hours of the day. In general, electricity is scarce around 10 and 18 and abundant during night hours. 

## E4

In the following we will run an experiment where we exogenously vary the capacity of the storage technology from 0 to 10. We will assume the storage duration of the technology is 4 hours.

We first define the two grids to solve the model over:

In [ ]:
# Setting
E2P = 4 # The "energy storage to power"-ratio/storage duration
maxcap = 10
loop = pd.Index(range(11), name = 'loop')
v0_ECap = pd.Series(0.001, index = getTechs(['Storage'],m.db))
vT_ECap = pd.Series(maxcap, index = v0_ECap.index)
v0_SCap = v0_ECap.mul(E2P)
vT_SCap = vT_ECap.mul(E2P)

grid_ECap = adjMultiIndex.addGrid(v0_ECap,vT_ECap,loop,'GeneratingCapacity').astype(int)
grid_SCap = adjMultiIndex.addGrid(v0_SCap,vT_SCap,loop,'sCap').astype(int)

We will also be extracting the following variables:

In [ ]:
extract = ['marginalSystemCosts','Generation','charge','discharge','capturePrice','marketValueFactor','capacityFactor','marginalEconomicValue','hourlyEmissions','Emissions','consumerSurplus','producerSurplus','Welfare']

We now run the counterfactual scenarios:

In [ ]:
solution = m.loopSolveExtract(loop, [grid_ECap, grid_SCap], extract)

__Explain what is happening to the marginal systems costs on an average day illustrate in the figure below.__

In [ ]:
Image(os.path.join(figs_path,'E2.4_sol.png'))

_Note: The code for generating the figure is:_

In [ ]:
%%capture
loop_id=loop[[1,3,6,10]]
# Make dataframe
df_p =  pd.concat([
    base_db['marginalSystemCosts'].rename('base').astype(float),
    adj.rc_pd(solution['marginalSystemCosts'],loop_id).unstack('loop').rename_axis(None,axis=1).astype(float)
],axis=1)

# Make new dataframe with time mappings
p_names = df_p.columns.copy()
df = df_p.reset_index().merge(m.db['h2HourOfTheDay'].to_frame(index=False))

# Make plot
fig, ax = plt.subplots()
df.groupby('HourOfTheDay')[p_names].mean().plot(xlabel='Average hour of the day',ylabel=r'Average marginal system costs ($€/$GJ)',ax=ax,legend=False)
fig.legend(['0 (baseline)']+grid_ECap[loop_id].tolist(),loc='lower center', ncol=len(df_fuelineff.columns), frameon=True, bbox_to_anchor=(0.6,0.2),title=r'Power capacity of storage technology:')
fig.tight_layout()
fig.savefig(f"{figs_path}\\E2.4_sol.png", edgecolor='k',bbox_inches='tight')

___
___Answer:___

Wee see that prices are generally increasing in low-price hours and decreasing in high-price hours. This is because the storage technology is charging in low-price states (demand increase pushing up prices) and dispatching in high-price states (supply increase pushing down prices). This can be illustrated by the graph below: 

In [ ]:
# Make dataframe
charge = adj.rc_pd(-solution['charge'].droplevel('id'),loop_id).unstack('loop').rename_axis(None,axis=1)
discharge = adj.rc_pd(solution['discharge'].droplevel('id'),loop_id).unstack('loop').rename_axis(None,axis=1)

df_Y =  pd.concat([charge,discharge],axis=0).groupby('h').agg('sum')
df_Y['base'] = 0

# Make new dataframe with time mappings
df = df_Y.reset_index().merge(m.db['h2HourOfTheDay'].to_frame(index=False))

# Make plot
fig, ax = plt.subplots()
df.groupby('HourOfTheDay')[p_names].mean().plot(xlabel='Average hour of the day',ylabel=r'$Y^d-Y^c$',ax=ax,legend=False)
fig.legend(['0 (baseline)']+grid_ECap[loop_id].tolist(),loc='lower center', ncol=len(df_fuelineff.columns), frameon=True, bbox_to_anchor=(0.6,0.2),title=r'Power capacity of storage technology:')
fig.tight_layout()

## E5

The cunk of code in the following illustrates the marginal economic value of both power capacity (left) and energy capacity (right) of storage using eq. __(4)__ and __(5)__, respectively, in Exercises 44-H.

__Explain why the marginal value of power capacity goes to zero and the marginal value of energy storage is still positive (although also decreasing)__.

In [ ]:
fig, ax = plt.subplots(ncols=2,sharey=True)
solution['marginalEconomicValue'].xs('idES',level='id').xs('Power',level='CapacityType').iloc[1:].plot(ax=ax[0],title='Marginal economic value of\npower capacity $(\partial \mathcal{L} /\partial q_{i})$',xlabel=r'Power capacity of storage technology')
solution['marginalEconomicValue'].xs('idES',level='id').xs('Energy',level='CapacityType').iloc[1:].plot(ax=ax[1],title='Marginal economic value of\nenergy capacity $(\partial \mathcal{L} /\partial \overline{S}_{i})$',xlabel=r'Power capacity of storage technology')
fig.tight_layout()

___
___Answer:___

The marginal economic value of power capacity is decreasing due to the indirect/equilibrium effect identified in eq. __(8)__ in the lecture note on electricity storage: Because the storage technology is eroding oppertunities of price arbitrage, marginally more price arbitration simply lowers the arbitrage opportunity. While this is an equilibrium effect a more conventional interpretation is that the storage technology features diminishing returns to power capacity. This effect implies that there are no short-run profits associated with investing in more power capacity above 5 GJ. 

However, we do see that the marginal value of energy storage is still positive. The intuition is that it would be optimal to invest in a storage technology with a longer storage duration than 4 hours. In other words, when the power capacity is at 5 GJ all arbitrage oppertunities within a typical day are eroded, and we would need a technology for long-term storage to take advantage of other price spreads. 

## E6

**Explain why total $CO_2$ emissions are increasing as illustrated by the folloing chunk of code:**

In [ ]:
solution['Emissions'].plot(ylabel='Ton CO$_2$',xlabel='Power capacity of storage')

___
___Answer:___

To answer this question, we need to look at what happens to emission in low-price states vs. high-price states:

In [ ]:
x = solution['hourlyEmissions'].xs('CO2',level='EmissionType').reset_index().merge(m.db['h2HourOfTheDay'].to_frame(index=False))
adj.rc_pd(x.groupby(['HourOfTheDay','loop'])[0].mean(),loop_id).mul(1000).unstack('loop').plot(ylabel='kg CO$_2$',xlabel='Hour of the day')

While we lowering emissions in high-price states in which stored electricity is dispatch in replacing fossil-based generation, we are crowding in emission in low-price states. Specifically, we are crowding in the coal- and natural-gas fired generators, which increases emissions on net due to their high emission intensity.